# 第一个Agent
- 会做数学题
- 不知道答案时可以搜索

In [10]:
from langchain_community.chat_models.tongyi import ChatTongyi
import os

api_key = os.getenv("KEY_TONGYI")
serp_api_key = os.getenv("KEY_SEARCH") #搜索平台Serp的API KEY
os.environ["SERPAPI_API_KEY"] = serp_api_key

# LLM
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0 # 0-1，越小越倾向与输入一致
)

## 搭建工具
- serpapi是一个聚合搜索引擎，需要安装谷歌搜索包以及申请账号 https://serpapi.com/
- llm-math是一个已经封装好的数学计算链

In [ ]:
#安装搜索包
! pip install google-search-results

In [5]:
from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["serpapi", "llm-math"], llm)


In [6]:
tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='fce7d0f24d2df568374ffd8a8eefbb2f368c71ba2ae7ff6e98d29733d2ebc8af', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='fce7d0f24d2df568374ffd8a8eefbb2f368c71ba2ae7ff6e98d29733d2ebc8af', aiosession=None)>),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a 

## 定义Agent

In [7]:
#定义agent, 使用小样本增强生成类型
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # 此处有不同类型
    verbose=True
)

e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
agent.run("现任美国总统是谁？他的年龄的平方是多少？")

e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
我需要先找到现任美国总统的名字，然后计算他的年龄的平方。
Action: Search
Action Input: "现任美国总统"
Observation: Joe Biden
Thought:我找到了答案，现任美国总统是Joe Biden。现在我需要计算他的年龄的平方。
Action: Calculator
Action Input: 78**2
Observation: Answer: 6084
Thought:我已计算出结果。
Final Answer: 现任美国总统是Joe Biden，他的年龄的平方是6084。

> Finished chain.


'现任美国总统是Joe Biden，他的年龄的平方是6084。'